# Import lib

In [4]:
from pyMSOO.MFEA.model import MFEA_base, SM_MFEA, LSA21
from pyMSOO.utils.Crossover import *
from pyMSOO.utils.Mutation import *
from pyMSOO.utils.Selection import *
from pyMSOO.utils.Search import * 
from pyMSOO.MFEA.benchmark.continous import *
from pyMSOO.utils.MultiRun.RunMultiTime import * 

from pyMSOO.utils.EA import * 
from pyMSOO.MFEA.benchmark.continous.CEC17 import CEC17_benchmark 
from pyMSOO.MFEA.benchmark.continous.WCCI22 import WCCI22_benchmark
from pyMSOO.MFEA.benchmark.continous.funcs import * 

from pyMSOO.utils.MultiRun.RunMultiTime import * 
from pyMSOO.utils.MultiRun.RunMultiBenchmark import * 

from pyMSOO.utils.LoadSaveModel.load_utils import loadModel, loadModelFromTxt
from pyMSOO.utils.LoadSaveModel.save_utils import export_history2txt

from pyMSOO.utils.numba_utils import *

# Run

## Define Tasks

In [2]:
# t, ic = CEC17_benchmark.get_10tasks_benchmark()

# ls_benchmark = [t]
# ls_IndClass = [ic]
# name_benchmark = ["cec17"]

# ls_benchmark = []
# ls_IndClass = []
# ls_tasks = [1]
# name_benchmark = [] 

# for i in ls_tasks:
#     # t, ic = WCCI22_benchmark.get_complex_benchmark(i)
#     t, ic = WCCI22_benchmark.get_50tasks_benchmark(i)
#     ls_benchmark.append(t)
#     ls_IndClass.append(ic)
#     name_benchmark.append(str(i))

ls_benchmark = []
ls_IndClass = []
ls_tasks = [1]
name_benchmark = [] 

for i in ls_tasks:
    # t, ic = WCCI22_benchmark.get_complex_benchmark(i)
    t, ic = WCCI22_benchmark.get_complex_benchmark(i)
    ls_benchmark.append(t)
    ls_IndClass.append(ic)
    name_benchmark.append(str(i))

## SM-MFEA

In [3]:
smpModel = MultiBenchmark(
    ls_benchmark= ls_benchmark,
    name_benchmark= name_benchmark,
    ls_IndClass= ls_IndClass,
    model= SM_MFEA
)

smpModel.compile( 
    crossover = SBX_Crossover(nc = 2),
    mutation = PolynomialMutation(nm = 5, pm= 1),
    selection= ElitismSelection(random_percent= 0),
    search= L_SHADE(len_mem= 15),
    attr_tasks = ['crossover', 'mutation', 'search'],
)
smpModel.fit(
    nb_generations= 1000, nb_inds_each_task= 100, nb_inds_min= 20,
    lr = 0.1, p_const_intra= 0., prob_search = 0., lc_nums = 200,
    nb_epochs_stop= 1000, swap_po= False,
    evaluate_initial_skillFactor= True
)
a = smpModel.run(
    nb_run= 1,     
    save_path= './RESULTS/'
)

Seed: 0 -- Time: 00m 35.49s  100 % [====================>]  Pop_size: 3.80E+01  ,  Cost: 6.06E+02  6.10E+02  ,  
[0.5 0.5]
[100086, 99920]
END!
DONE!
Saved


### Load Model

In [ ]:
model_algorithms = loadModel(
    PATH= "/home/anhdt-minhdt/Data/CEC17/SBX/S-MFEA_CEC17_PYTHON.mso",
    ls_tasks= CEC17_benchmark.get_10tasks_benchmark()[0]
    )

Error while create instance model for <class 'pyMSOO.utils.EA.Population'>. Using Temp class instead
Error while create instance model for <class 'pyMSOO.utils.EA.SubPopulation'>. Using Temp class instead
Error when create object DictType. Using Temp class instead
Error when create object UnicodeType. Using Temp class instead
Error when create object Float. Using Temp class instead
Error when create object Tuple. Using Temp class instead
Error when create object UnionType. Using Temp class instead


In [4]:
model = loadModelFromTxt(
    source_path= "./RESULTS/MTOSOO_P1.txt",  
    model_algorithm= SM_MFEA, 
    multitime_model_class= MultiTimeModel, 
    target_path= "./RESULTS/", 
    remove_tasks= True, 
    history_cost_shape= (1000, 2), 
    nb_runs= 1,
    ls_tasks= WCCI22_benchmark.get_complex_benchmark(1)[0],

)

In [6]:
model.history_cost[-1]

array([605.513152, 610.187823])

In [ ]:
a = np.random.rand(100, 100)

In [ ]:
model_algorithms.render_history(yscale='log')

## Convert

In [5]:
import pickle 
import os 

def export2txt(source_file, 
               save_file, 
               total_evals = int(100000 * 2), 
                steps = 1000, ):
        model = pickle.load(open(source_file, 'rb'))
        tmp = np.concatenate([model['ls_model'][i]['history_cost'] for i in range(len(model['ls_model']))], axis=1)
        stt = np.arange(total_evals // steps, total_evals + total_evals // steps, total_evals // steps)
        # stt = stt.reshape(1,int((200000)/ 2000))
        stt = stt.reshape(1, -1) 
        tmp = tmp.T
        if tmp.shape[0] != stt.shape[1]:
            tmp = tmp[:, -stt.shape[1]:]
        
        assert tmp.shape[1] == stt.shape[1], print(tmp.shape)
        tmp4= np.concatenate([stt, tmp], axis=0).T

        f = open(save_file, "w")
        for line in tmp4: 
            f.write(", ".join([str(int(line[0]))] + [str('{:.6f}'.format(e)) for e in line[1:]]) + "\n")
        f.close()

def export_all2txt(source_folder, destination_folder): 
    for item in os.listdir(source_folder): 
        current_source_path= os.path.join(source_folder, item)
        if os.path.isdir(current_source_path):
            export_all2txt(current_source_path, os.path.join(destination_folder, item))
        else: 
            current_save_path = os.path.join(destination_folder, item)[:-4] + ".txt"
            if os.path.isdir(destination_folder) is False: 
                os.makedirs(destination_folder) 
            
            file = pickle.load(open(current_source_path, 'rb'))
            export2txt(
                 source_file= current_source_path, 
                 save_file= current_save_path, 
                 total_evals= int(1e5 * 50), 
                 steps= 1000, 
            )

def export_all2mso(source_folder, destination_folder): 
    for item in os.listdir(source_folder): 
        current_source_path= os.path.join(source_folder, item)
        if os.path.isdir(current_source_path):
            export_all2txt(current_source_path, os.path.join(destination_folder, item))
        else: 
            current_save_path = os.path.join(destination_folder, item)[:-4] + ".mso"
            if os.path.isdir(destination_folder) is False: 
                os.makedirs(destination_folder) 
            
            # file = pickle.load(open(current_source_path, 'rb'))
            model = loadModelFromTxt(
                source_path= current_source_path, 
                model_algorithm= AbstractModel, 
                multitime_model_class= MultiTimeModel, 
                target_path= "./", 
                history_cost_shape= (1000, 50), 
                nb_runs= 30, 
                ls_tasks= WCCI22_benchmark.get_50tasks_benchmark(1)[0], 
            )
            saveModel(
                model= model, 
                PATH= current_save_path, 
                remove_tasks= True, 
            )


            

In [2]:
import os 
len(os.listdir("/home/anhdt-minhdt/Data/GECCO/"))

11

In [ ]:
export_all2txt(source_folder= "/home/anhdt-minhdt/Data/GECCO/", 
               destination_folder= "./RESULTS/GECCO/")

In [12]:
export_all2mso(
    source_folder= "./RESULTS/GECCO/", 
    destination_folder= "./RESULTS/GECOO_mso/"
)

UnpicklingError: invalid load key, '5'.

In [8]:
model = loadModel("RESULTS/GECOO_mso/MFEA_KL/1.mso", ls_tasks= WCCI22_benchmark.get_50tasks_benchmark(2)[0])

In [10]:
model.history_cost[-1]

array([0.8522303 , 0.9419645 , 0.96797887, 0.94098893, 0.8318025 ,
       0.92539443, 0.96628533, 0.98798047, 0.99480053, 0.9444166 ,
       0.92271843, 0.9660901 , 1.02869953, 0.8659094 , 1.02584903,
       0.9281827 , 1.02069687, 0.93335473, 0.90112797, 0.891759  ,
       1.00860887, 1.05985147, 1.02016023, 0.94743387, 0.98020567,
       1.04663797, 1.07077087, 1.0146281 , 1.00286147, 1.09057673,
       1.187477  , 1.01144677, 1.0756147 , 1.09476837, 1.1062548 ,
       1.13935483, 1.03186513, 1.09499177, 1.08249047, 1.25032623,
       1.11671147, 1.17827177, 1.20219317, 1.1992211 , 1.2499264 ,
       1.29190693, 1.16024347, 1.17683137, 1.2354259 , 1.25938863])

In [3]:
file = pickle.load(open("/home/anhdt-minhdt/Data/GECCO/EBS_GA/EBS_GA_Benchmark_10.mso", 'rb'))
file['ls_model'][0]['history_cost']

array([[1.17516583e-01, 3.61479165e+02, 6.53091458e-04, ...,
        6.61936836e-04, 5.58515814e+00, 1.02903667e+04],
       [1.17516583e-01, 3.61479165e+02, 6.53091458e-04, ...,
        6.61936836e-04, 5.58515814e+00, 1.02903667e+04],
       [1.17516583e-01, 3.61479165e+02, 6.53091458e-04, ...,
        6.61936836e-04, 5.58515814e+00, 1.02903667e+04],
       ...,
       [1.17516583e-01, 3.61479165e+02, 6.53091458e-04, ...,
        6.61936836e-04, 5.58515814e+00, 1.02903667e+04],
       [1.17516583e-01, 3.61479165e+02, 6.53091458e-04, ...,
        6.61936836e-04, 5.58515814e+00, 1.02903667e+04],
       [1.17516583e-01, 3.61479165e+02, 6.53091458e-04, ...,
        6.61936836e-04, 5.58515814e+00, 1.02903667e+04]])